In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

In [3]:
!pip install -q pandas==1.1.5

     |████████████████████████████████| 9.5 MB 17.4 MB/s 


In [4]:
%sql sqlite:///RealWorldData.db



'Connected: @RealWorldData.db'

In [6]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


In [14]:
%%sql
select
 count(*)
from
  CHICAGO_CRIME_DATA

 * sqlite:///RealWorldData.db
Done.


count(*)
533


In [20]:
%%sql
select
 community_area_number,
 community_area_name,
 per_capita_income
from
  CENSUS_DATA
where
  per_capita_income < 11000

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


In [27]:
%%sql
select
  id,
  case_number, 
  primary_type,
  description
from
  CHICAGO_CRIME_DATA
where
  description like '%minor%'

 * sqlite:///RealWorldData.db
Done.


ID,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
3987219,HL266884,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR
3266814,HK238408,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR


In [28]:
%%sql
select
  id,
  case_number,
  primary_type,
  description
from
  CHICAGO_CRIME_DATA
where
  primary_type like '%kidnapping%'
  and
  description like '%child%'

 * sqlite:///RealWorldData.db
Done.


ID,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
5276766,HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


In [42]:
%%sql
select
  distinct primary_type, 
  /* description ,*/
  location_description
from
  CHICAGO_CRIME_DATA
where
  location_description like '%school%'
order by
  primary_type

 * sqlite:///RealWorldData.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


In [49]:
%%sql
select
  "Elementary, Middle, or High School",
  avg(safety_score)
from
  CHICAGO_PUBLIC_SCHOOLS_DATA
group by  
  "Elementary, Middle, or High School"


 * sqlite:///RealWorldData.db
Done.


"Elementary, Middle, or High School",avg(safety_score)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [58]:
%%sql
select
  community_area_name,
  max(percent_households_below_poverty) as highest_percent_households_below_poverty
from
  census_data
where
  percent_households_below_poverty is not null
group by 
  community_area_name
order by 
  highest_percent_households_below_poverty desc
limit 5


 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,highest_percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [69]:
%%sql
select
  community_area_number,
  count(community_area_number) as CrimeNumber
from
  CHICAGO_CRIME_DATA
where
  community_area_number is not null
group by  
  community_area_number
order by 
  crimenumber desc
limit 1

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,CrimeNumber
25.0,43


In [71]:
%%sql
select
  community_area_number,
  community_area_name,
  hardship_index
from
  census_data
where
  hardship_index = (
    select max(hardship_index)
    from census_data
  )

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
54.0,Riverdale,98.0


In [75]:
%%sql
select
  distinct ccd.community_area_number, cd.community_area_name
from
  CHICAGO_CRIME_DATA ccd
join 
  census_data cd
on  
  ccd.community_area_number = cd.community_area_number
where
  ccd.community_area_number = (
    select community_area_number
    from CHICAGO_CRIME_DATA
    where community_area_number is not null
    group by community_area_number
    order by count(community_area_number) desc 
    limit 1
  )


 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin
